<a href="https://colab.research.google.com/github/reljadev/DeepLearning/blob/main/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
from torch import nn
from torch import optim
from torch import Tensor
import dlc_practical_prologue as prologue

In [59]:
class ConvNet(nn.Module):

  def __init__(self, nb_channels, kernel_size = 3):
    super().__init__()
    self.nb_channels = nb_channels

    # 2 x 14 x 14
    self.conv1 = nn.Conv2d(in_channels=2, 
                           out_channels=nb_channels,
                           kernel_size=kernel_size,
                           padding = (kernel_size - 1) // 2)
    # nb_channels x 14 x 14
    self.bn1 = nn.BatchNorm2d(nb_channels)
    # nb_channels x 14 x 14
    self.pool1 = nn.MaxPool2d(kernel_size=2)

    # nb_channels x 7 x 7
    self.lin1 = nn.Linear(in_features=nb_channels*7*7,
                          out_features=1024
                         )
    # 1024
    self.lin2 = nn.Linear(in_features=1024,
                          out_features=1
                         )
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
  

  def forward(self, x):
    x = self.pool1(self.bn1(self.conv1(x)))
    x = x.view(-1, self.nb_channels*7*7)
    x = self.relu(self.lin1(x))
    x = self.sigmoid(self.lin2(x))
    if not self.training:
      x = 1 * (x > 0.5)

    return x.view(-1)

In [58]:
def train_model(model, train_input, train_target):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    nb_epochs = 20
    mini_batch_size = 100

    model.train()
    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            target = train_target.narrow(0, b, mini_batch_size).float()
            loss = criterion(output, target)
            model.zero_grad()
            loss.backward()
            optimizer.step()

In [52]:
def compute_nb_errors(model, data_input, data_target):
  nb_data_errors = 0
  mini_batch_size = 100

  model.eval()
  for b in range(0, data_input.size(0), mini_batch_size):
    input = data_input.narrow(0, b, mini_batch_size)
    output = model(input)
    target = data_target.narrow(0, b, mini_batch_size)
    nb_data_errors += torch.sum(torch.abs(output - target))

  return nb_data_errors

In [4]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(nb=1000)

In [63]:
model = ConvNet(nb_channels=32, kernel_size=3)
train_model(model, train_input, train_target)
print("Error: {:.2f}%".format((compute_nb_errors(model, test_input, test_target) / 1000).item() * 100))

Error: 17.60%
